# Snapshot Localization Algorithms

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Stanford-NavLab/gnss_lib_py/blob/main/notebooks/tutorials/algorithms/snapshot.ipynb)

These tutorials demonstrate the snapshot localization algorithms available in ``gnss_lib_py``.

## Weighted Least Squares

In [ ]:
import gnss_lib_py as glp

In [ ]:
# load Android Google Challenge data
glp.make_dir("../data")
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/google_decimeter_2021/Pixel4XL_derived.csv --quiet -nc -O "../data/Pixel4XL_derived.csv"
derived_data = glp.AndroidDerived2021("../data/Pixel4XL_derived.csv", remove_timing_outliers=False)

Solve for the Weighted Least Squares position estimate simply by passing the measurement data.

When obtaining WLS estimates for real measurements, the rotation of the Earth between the signal transmission and reception has to be accounted for.
`solve_wls` accounts for this by default and rotates the given SV positions into the ECEF frame of reference when the signals were received rather using the ECEF frame of reference of when the signals were transmitted.

If you assume that the satellite positions are given in the ECEF frame of reference when the signals were received (and not transmitted), set the parameter `sv_rx_time = True` in the function call.

In [ ]:
state_wls = glp.solve_wls(derived_data)
# When assuming that SV positions are given in the ECEF frame when signals are received use
# state_wls = glp.solve_wls(derived_data, sv_rx_time=True)

Plot the ECEF x and ECEF y computed position estimate of the receiver

In [ ]:
glp.plot_map(state_wls)

## Custom Weighting Schemes for WLS

You can utilize custom weights for each measurement by passing in a row name into the ``weight_type`` parameter.

In [ ]:
# Morton, Y. Jade, et al., eds. Position, navigation, and timing technologies in
# the 21st century: Integrated satellite navigation, sensor systems, and civil
# applications, volume 1. John Wiley & Sons, 2021. Section 11.3.1.
derived_data["weights"] = 1./derived_data["raw_pr_sigma_m"]**2

state_wls_sigma = glp.solve_wls(derived_data,weight_type="weights")

state_wls_sigma.rename({"lat_rx_wls_deg":"lat_rx_" + "sigma" + "_deg",
                        "lon_rx_wls_deg":"lon_rx_" + "sigma" + "_deg",
                         "alt_rx_wls_m":"alt_rx_" + "sigma" + "_m",
                         }, inplace=True)

In [ ]:
glp.plot_map(state_wls, state_wls_sigma)

## Estimating Only Receiver Clock Bias with WLS

If you only need to estimate the receiver clock bias, you can set the ``only_bias`` parameter to ``True`` and pass in the receiver position in with the ``receiver_state`` parameter.

In [ ]:
state_with_clock_bias = glp.solve_wls(derived_data, only_bias=True, receiver_state=state_wls)

In [ ]:
fig = glp.plot_metric(state_with_clock_bias,"gps_millis","b_rx_wls_m")